# Modelo para la predicción de precios de viviendas en Melbourne

A partir de los datos de una serie de viviendas de Melbourne, con sus respectivos precios de venta, se va a tratar de elegir un modelo para la predicción del precio en función de las características del piso

In [175]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
%matplotlib inline
pd.set_option('display.max_columns', 22)


In [34]:
melb_df = pd.read_csv('melb_data.csv')
display(melb_df.head())

,Unnamed: 0,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,5,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [35]:
display('All columns: ', melb_df.columns)
display('Numeric columns: ', melb_df._get_numeric_data().columns)

'All columns: '

Index(['Unnamed: 0', 'Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method',
       'SellerG', 'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom',
       'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea',
       'Lattitude', 'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

'Numeric columns: '

Index(['Unnamed: 0', 'Rooms', 'Price', 'Distance', 'Postcode', 'Bedroom2',
       'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude',
       'Longtitude', 'Propertycount'],
      dtype='object')

In [36]:
#Comprobamos el número de nulos por columna y comparamos con los datos totales
display(melb_df.shape)
display(melb_df.isnull().sum().sort_values(ascending = False))

(18396, 22)

BuildingArea     10634
YearBuilt         9438
CouncilArea       6163
Landsize          4793
Car               3576
Bathroom          3471
Bedroom2          3469
Longtitude        3332
Lattitude         3332
Propertycount        1
Distance             1
Postcode             1
Regionname           1
Date                 0
SellerG              0
Method               0
Price                0
Type                 0
Rooms                0
Address              0
Suburb               0
Unnamed: 0           0
dtype: int64

Comprobamos que para las columnas BuildingArea, YearBuilt y CouncilArea, el total de nulos es muy elevado con respecto al número total de registros, por lo que desestimamos esas columnas y eliminamos el resto de filas que contienen nulos. También se elimina la columna Unnamed: 0.

A continuación comprobamos los registros que nos quedan:

In [37]:
melb_df.drop(['Unnamed: 0','BuildingArea', 'YearBuilt', 'CouncilArea'], axis = 1, inplace = True)
melb_df.dropna(inplace = True)
display(melb_df.shape)
display(melb_df.isnull().sum().sort_values(ascending = False))

(13518, 18)

Propertycount    0
Regionname       0
Address          0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
Lattitude        0
Longtitude       0
Suburb           0
dtype: int64

In [39]:
display(melb_df.Suburb.unique())
display(melb_df.Type.unique())
display(melb_df.Method.unique())
display(melb_df.Postcode.unique())
display(melb_df.Regionname.unique())

array(['Abbotsford', 'Airport West', 'Albert Park', 'Alphington',
       'Altona', 'Altona North', 'Armadale', 'Ascot Vale', 'Ashburton',
       'Ashwood', 'Avondale Heights', 'Balaclava', 'Balwyn',
       'Balwyn North', 'Bentleigh', 'Bentleigh East', 'Box Hill',
       'Braybrook', 'Brighton', 'Brighton East', 'Brunswick',
       'Brunswick West', 'Bulleen', 'Burwood', 'Camberwell', 'Canterbury',
       'Carlton North', 'Carnegie', 'Caulfield', 'Caulfield North',
       'Caulfield South', 'Chadstone', 'Clifton Hill', 'Coburg',
       'Coburg North', 'Collingwood', 'Doncaster', 'Eaglemont',
       'Elsternwick', 'Elwood', 'Essendon', 'Essendon North', 'Fairfield',
       'Fitzroy', 'Fitzroy North', 'Flemington', 'Footscray', 'Glen Iris',
       'Glenroy', 'Gowanbrae', 'Hadfield', 'Hampton', 'Hampton East',
       'Hawthorn', 'Heidelberg Heights', 'Heidelberg West', 'Hughesdale',
       'Ivanhoe', 'Kealba', 'Keilor East', 'Kensington', 'Kew',
       'Kew East', 'Kooyong', 'Maidstone', 

array(['h', 'u', 't'], dtype=object)

array(['S', 'SP', 'PI', 'VB', 'SA'], dtype=object)

array([3067., 3042., 3206., 3078., 3018., 3025., 3143., 3032., 3147.,
       3034., 3183., 3103., 3104., 3204., 3165., 3128., 3019., 3186.,
       3187., 3056., 3055., 3105., 3125., 3124., 3126., 3054., 3163.,
       3162., 3161., 3148., 3068., 3058., 3066., 3108., 3084., 3185.,
       3184., 3040., 3041., 3065., 3031., 3011., 3146., 3046., 3043.,
       3188., 3122., 3081., 3166., 3079., 3021., 3033., 3101., 3102.,
       3144., 3012., 3145., 3000., 3127., 3039., 3189., 3015., 3051.,
       3070., 3167., 3052., 3044., 3207., 3181., 3072., 3073., 3121.,
       3205., 3141., 3006., 3182., 3020., 3107., 3071., 3142., 3087.,
       3003., 3016., 3085., 3013., 3057., 3061., 3053., 3002., 3060.,
       3123., 3047., 3083., 3008., 3153., 3193., 3806., 3155., 3088.,
       3023., 3151., 3192., 3064., 3977., 3136., 3175., 3089., 3172.,
       3109., 3111., 3754., 3095., 3076., 3131., 3199., 3200., 3437.,
       3803., 3777., 3190., 3037., 3038., 3173., 3075., 3093., 3337.,
       3082., 3132.,

array(['Northern Metropolitan', 'Western Metropolitan',
       'Southern Metropolitan', 'Eastern Metropolitan',
       'South-Eastern Metropolitan', 'Eastern Victoria',
       'Northern Victoria', 'Western Victoria'], dtype=object)

Comprobamos que hay un número elevado de suburbios y de códigos postales, pero el número de tipos (Type), nombre de región y métodos (method) es más reducido, por lo que consideramos generar columnas dummies para estos campos.

A continuación pasamos a seleccionar las columnas que más nos interesan:

In [45]:
selected_columns = ['Rooms', 'Type', 'Price', 'Method', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'Lattitude', 'Longtitude', 'Regionname']
melb_df = melb_df[selected_columns]

In [46]:
melb_df.head()

,Rooms,Type,Price,Method,Bedroom2,Bathroom,Car,Landsize,Lattitude,Longtitude,Regionname
0,2,h,1480000.0,S,2.0,1.0,1.0,202.0,-37.7996,144.9984,Northern Metropolitan
1,2,h,1035000.0,S,2.0,1.0,0.0,156.0,-37.8079,144.9934,Northern Metropolitan
2,3,h,1465000.0,SP,3.0,2.0,0.0,134.0,-37.8093,144.9944,Northern Metropolitan
3,3,h,850000.0,PI,3.0,2.0,1.0,94.0,-37.7969,144.9969,Northern Metropolitan
4,4,h,1600000.0,VB,3.0,1.0,2.0,120.0,-37.8072,144.9941,Northern Metropolitan


In [47]:
melb_df = pd.get_dummies(melb_df, columns = ['Type', 'Method', 'Regionname'], drop_first=True)

Una vez que tenemos los datos limpios, con todos los campos en estado numérico, seleccionamos las features y el target, que será el precio del piso.

In [50]:
X = melb_df.drop('Price', axis = 1)
y = melb_df.Price

In [155]:
robust_scaler = RobustScaler()
X_scaled = robust_scaler.fit(X).transform(X)

In [156]:
train_X, val_X, train_y, val_y = train_test_split(X_scaled, y, test_size=0.2)

In [163]:
decisionTree = DecisionTreeRegressor(random_state=1).fit(train_X, train_y)
val_predictions = rforest.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
display("Error medio absoluto/precio medio de las viviendas: {:.2f}".format(val_mae/np.mean(y)*100))
display("Score obtenido por el modelo: {:.2f}". format(rforest.score(val_X, val_y)))

'Error medio absoluto/precio medio de las viviendas: 21.17'

'Score obtenido por el modelo: 0.59'

In [165]:
rforest = RandomForestRegressor(random_state=1).fit(train_X, train_y)
val_predictions = rforest.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
display("Error medio absoluto/precio medio de las viviendas: {:.2f}".format(val_mae/np.mean(y)*100))
display("Score obtenido por el modelo: {:.2f}". format(rforest.score(val_X, val_y)))

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


'Error medio absoluto/precio medio de las viviendas: 16.72'

'Score obtenido por el modelo: 0.78'

In [99]:
score = rforest2.score(val_X_scaled, val_y_log)

In [100]:
score

0.5543279692373921

In [183]:
def rmse_cv(model, X, y, valX, valy):
    error = mean_absolute_error(val_predictions, val_y)/np.mean(y)*100
    return [model.fit(X, y).score(valX, valy), error]

In [ ]:
models = [DecisionTreeRegressor(), RandomForestRegressor(),
          LinearRegression(), Lasso(alpha=0.01,max_iter=10000),
          SGDRegressor(max_iter=1000),
          KernelRidge(alpha=0.6,kernel='polynomial',degree = 2,coef0=2.5),
          ]
names = ['DT','RF', 'LR', 'Lasso', 'SGDR','KR']
for model,name in zip(models,names):
    score = rmse_cv(model,train_X, train_y, val_X, val_y)
    print("Error medio absoluto/precio medio de las viviendas: {:.2f}".format(score[1]))
    print("Score obtenido por el modelo {}: {:.6f}".format(name,score[0]))

Error medio absoluto/precio medio de las viviendas: 16.74
Score obtenido por el modelo DT: 0.594405


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Error medio absoluto/precio medio de las viviendas: 16.74
Score obtenido por el modelo RF: 0.793127
Error medio absoluto/precio medio de las viviendas: 16.74
Score obtenido por el modelo LR: 0.520197
Error medio absoluto/precio medio de las viviendas: 16.74
Score obtenido por el modelo Lasso: 0.520197


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Error medio absoluto/precio medio de las viviendas: 16.74
Score obtenido por el modelo SGDR: -245.442115
